Following classification model classifies bees and ants using resnet50 pretrained on imagenet dataset. The following code is in reference to the computer vision transfer learning tutorial from pytorch documentation.

Importing the required dependencies

In [ ]:
from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy

Data augmentation usind torchvision.transforms class provided by pytorch.
We are cropping the image to 224 px as the pretrained model accepts the minimum size of 224 px, horizontally flipping it and converting into tensors followed by normalizing the tensors.


In [ ]:
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

The bees vs ants data was loaded into the drive which is mounted to pipe the data into the dataloaders. We are iterating through the dataset to apply transformations to the images and loading the classes.

In [ ]:
data_dir = '/content/drive/My Drive/Data/hymenoptera_data'
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          data_transforms[x])
                  for x in ['train', 'val']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=4,
                                             shuffle=True, num_workers=4)
              for x in ['train', 'val']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
class_names = image_datasets['train'].classes

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

The following code is for training the model, we copy the model weights in best_model_wts and compare it with the weights for other iterations. The set of weights which give the highest accuracy are used for the final model.

In [ ]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=8):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

       
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  
            else:
                model.eval()   

            running_loss = 0.0
            running_corrects = 0

           
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

               
                optimizer.zero_grad()

               
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    
    model.load_state_dict(best_model_wts)
    return model


The following code imports the resnet50 pretrained model from torchvision.models, we do not apply the gradient and add a linear layer in the end to get a binary output. We apply stochastic gradient descent only on the fully connected layer. We are also using the lr_sceduler to decay the learning rate.

In [ ]:
model_conv = torchvision.models.resnet50(pretrained=True)
for param in model_conv.parameters():
    param.requires_grad = False


num_ftrs = model_conv.fc.in_features
model_conv.fc = nn.Linear(num_ftrs, 2)

model_conv = model_conv.to(device)

criterion = nn.CrossEntropyLoss()


optimizer_conv = optim.SGD(model_conv.fc.parameters(), lr=0.001, momentum=0.9)


exp_lr_scheduler = lr_scheduler.StepLR(optimizer_conv, step_size=7, gamma=0.1)

Training the model.

In [ ]:
model_conv = train_model(model_conv, criterion, optimizer_conv,
                         exp_lr_scheduler, num_epochs=8)

Epoch 0/7
----------
train Loss: 0.6757 Acc: 0.6639
val Loss: 0.6022 Acc: 0.6797

Epoch 1/7
----------
train Loss: 0.5492 Acc: 0.7664
val Loss: 0.2393 Acc: 0.9085

Epoch 2/7
----------
train Loss: 0.4973 Acc: 0.7746
val Loss: 0.2580 Acc: 0.8824

Epoch 3/7
----------
train Loss: 0.4623 Acc: 0.7787
val Loss: 0.2175 Acc: 0.9085

Epoch 4/7
----------
train Loss: 0.2480 Acc: 0.8975
val Loss: 0.2853 Acc: 0.8758

Epoch 5/7
----------
train Loss: 0.4520 Acc: 0.8156
val Loss: 0.2301 Acc: 0.9281

Epoch 6/7
----------
train Loss: 0.3418 Acc: 0.8607
val Loss: 0.1663 Acc: 0.9346

Epoch 7/7
----------
train Loss: 0.2894 Acc: 0.8648
val Loss: 0.1511 Acc: 0.9346

Training complete in 12m 27s
Best val Acc: 0.934641


Saving the model using torch.save

In [ ]:
PATH = '/content/drive/My Drive/BEES_ANTS.pt'
torch.save(model_conv, PATH)

In [ ]:
#model_conv = torch.load(PATH)
model_conv.eval()

The following code helps to  visualize the performance of the model. We are loading some images from the validation dataset and passing through the model and displaying the obtained result.

In [ ]:
def visualize_model(model, num_images=5):
    was_training = model.training
    model.eval()
    images_so_far = 0
    fig = plt.figure()

    with torch.no_grad():
        for i, (inputs, labels) in enumerate(dataloaders['val']):
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)

            for j in range(inputs.size()[0]):
                images_so_far += 1
                ax = plt.subplot(num_images//2, 2, images_so_far)
                ax.axis('off')
                ax.set_title('predicted: {}'.format(class_names[preds[j]]))
                imshow(inputs.cpu().data[j])

                if images_so_far == num_images:
                    model.train(mode=was_training)
                    return
        model.train(mode=was_training)

Passing model_conv to visualize_model function.

In [ ]:
visualize_model(model_conv)

plt.ioff()
plt.show()